# Simulation QCQP - equivalent QP

<img src="http://media.giphy.com/media/elvaNxcznR8Fa/giphy.gif"  width="200">


In [1]:
import cvxpy as cvx
import numpy as np
import matplotlib.pyplot as plt
from qcqp import *
from numpy.random import randn


## Plan :

   [- QCQP](#1)
   
   [- QP](#2)
   
   [- QP vs. QCQP](#3)   

<a id="1"></a> 
 
# 1. QCQP

## Functions

Function isIncluded with QCQP algorithm

In [153]:
def IsIncluded_QCQP(cI,RI,cU,RU,n,p,q):
    i = 0
    response = False
    x = cvx.Variable(n)
    while i < q: 
        objective = -sum((x-cU[i])**2)
        constraints  = [sum((x-cI[0])**2) <= RI[0]**2]
        for k in range(1, p):
            constraints = constraints + [sum((x-cI[k])**2) <= RI[k]**2]
        for l in range(0, i):
            constraints = constraints + [sum((x-cU[l])**2) >= RU[l]**2]

        prob = cvx.Problem(cvx.Minimize(objective), constraints)  
        qcqp = QCQP(prob)
        qcqp.suggest(RANDOM)  
        #qcqp.suggest(SDR,  solver=cvx.CVXOPT)
        #qcqp.suggest(SPECTRAL)
        #f_cd = qcqp.improve(COORD_DESCENT) 
        f_cd = qcqp.improve(ADMM) 
        
        if -f_cd[0] <= RU[i]**2: 
            response = True
            i = q  
            
        i = i+1
    return response

Function IsIncluded using plot 2D

In [154]:
def IsIncluded_PLOT(cI,RI,cU,RU,p,q):
    h = .1  # step size in the mesh
    x_min, x_max =  -30, 30
    y_min, y_max = -30, 30
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),np.arange(y_min, y_max, h))
    i = 0
    while i < len(xx):
        j = 0
        while j < len(yy):
            response = True
            for k in range(0,p):
                if((xx[i,j]-cI[k,0])**2+(yy[i,j]-cI[k,1])**2>RI[k]**2):
                    response = False
                    k = p
            if response == True:
                for l in range(0,q):
                    if((xx[i,j]-cU[l,0])**2+(yy[i,j]-cU[l,1])**2<=RU[l]**2):
                        response = False
                        l = q

            if response == True:
                #print(xx[i,j],yy[i,j])
                i = len(xx)
                j = len(yy)
            j= j+1
        i=i+1
    return not response

Function topology

In [155]:
def topology(cI,RI,cU,RU,n,p,q):
    c = np.concatenate((cI, cU), axis=0)
    R = np.concatenate((RI, RU), axis=0)
    response = np.ones((p+q, p+q))
    for i in range(0,p+q):
        for j in range(0,p+q):
            dist = np.sqrt(sum((c[i]-c[j])*(c[i]-c[j])))
            if dist > R[i]+R[j] : 
                response[i,j] = 0
            if dist < R[j]-R[i] : 
                response[i,j] = 2
            if dist < R[i]-R[j] : 
                response[i,j] = 3
                
    return response

Function plot 2D

In [181]:
def plot(cI,RI,cU,RU,p,q, scale = 30):
    fig, ax = plt.subplots() 
    ax.set_xlim((-scale, scale))
    ax.set_ylim((-scale, scale))

    for i in range(0, q):
        ax.add_artist(plt.Circle(list(cU[i]), RU[i], color='r',  alpha=1))
    for i in range(0, p):
        ax.add_artist(plt.Circle(list(cI[i]), RI[i], color='b', alpha=.15))

Function data generator

In [157]:
def dataGenerator(epsilon,n,p,q,diff):
    rand = np.random.normal(0, 10, n)
    cI = rand.reshape((1, n))
    RI = np.sqrt((cI*cI).sum(axis=1)) + epsilon
    
    i = 0   
    while i < p+q:
        if i < p:
            rand = np.random.normal(0, 10, n)
            c = rand.reshape((1, n))
            R = np.sqrt((c*c).sum(axis=1)) + epsilon
        if i >= p:
            rand = np.random.normal(0, 10*diff, n)
            c = rand.reshape((1, n))
            R = np.sqrt((c*c).sum(axis=1)) + epsilon/2
        concat = True
        
        for j in range(0,cI.shape[0]):
            dist = np.linalg.norm((cI[j] - c), ord=1)
            if dist < abs(RI[j]-R):
                concat = False
        
        if concat == True:
            cI = np.concatenate((cI, c), axis=0)
            RI = np.concatenate((RI, R), axis=0)
            i = i + 1
    cU = cI[0:q,]
    cI = cI[q+1:p+q+1,] 
    RU = RI[0:q]
    RI = RI[q+1:p+q+1]     
    return(cI,RI,cU,RU)

## Simple test

In [158]:
epsilon = 5
#n = nb of varialbles
n = 2
#p = nb of elements in the intersection
p = 5
#p = nb of elements in the union
q = 2

In [159]:
rand = np.random.normal(0, 10, p*n)
cI = rand.reshape((p, n))
RI = np.sqrt((cI*cI).sum(axis=1)) + epsilon

rand = np.random.normal(0, 10, q*n)
cU = rand.reshape((q, n))
RU = np.sqrt((cU*cU).sum(axis=1)) + epsilon/2

In [ ]:
IsIncluded_QCQP(cI,RI,cU,RU,n,p,q)

In [ ]:
IsIncluded_PLOT(cI,RI,cU,RU,p,q)

In [ ]:
topology(cI,RI,cU,RU,n,p,q)

In [ ]:
plot(cI,RI,cU,RU,p,q)

<img src="http://media.giphy.com/media/bvARFeSkNzIm4/giphy.gif"  width="300">

<a id="2"></a> 
 
# 2. QP

## Functions

In [164]:
from cvxopt import matrix, solvers
import cdd 
import mosek

In [165]:
def IsIncluded_QP(cI,RI,cU,RU,n,p,q):
    i = 0
    response = True
    x = cvx.Variable(n)
    while i < q: 
        #Create constraints G*x <= h
        cU2 = np.delete(cU, (i), axis=0)
        RU2 = np.delete(RU, (i), axis=0)
        h1 = RI**2 - RU[i]**2 + sum(cU[i]**2) - (cI**2).sum(axis=1)
        h2 = RU[i]**2 - RU2**2 - sum(cU[i]**2) + (cU2**2).sum(axis=1)
        h = np.concatenate((h1,h2), axis=0)
        G1 = 2*(cU[i] - cI)
        G2 = 2*(cU2 - cU[i])
        G = np.matrix(np.concatenate((G1,G2), axis=0))

        # Create variable.
        x = cvx.Variable(n)
        # Pb solution.    
        constraints = [G*x <= h]
        obj = cvx.Minimize(sum((x-cU[i])**2))
        prob = cvx.Problem(obj, constraints)
        
        if prob.solve() <RU[i]**2: #Study of the intersection circle - polyhedron   
            hh = h.reshape(G.shape[0], 1) 
            m = np.asarray(np.concatenate((hh,-G), axis=1))
            mat = cdd.Matrix(m, number_type='float')
            mat.rep_type = cdd.RepType.INEQUALITY
            poly = cdd.Polyhedron(mat)
            result = np.array(poly.get_generators())
            #print(result)
            if result.shape[0]>0:
                if (all(result[:,0] == 1) == True):#polyhedron = polytope
                    for j in range(0,result.shape[0]):
                        if (sum((result[j,1:]-cU[i])**2) > RU[i]**2):#vertex outside the ball
                            response = False
                else:
                    response = False #unbounded polyhedron
        i = i+1    
        if response == False:
            i = q
    
    if response == True: #case: partial U intersection I = emptyset
        objective = sum((x-cI[p-1])**2)
        constraints  = [sum((x-cI[0])**2) <= RI[0]**2]
        for k in range(1, p-1):
            constraints = constraints + [sum((x-cI[k])**2) <= RI[k]**2]
        probINT = cvx.Problem(cvx.Minimize(objective), constraints)  
        qcqp = QCQP(probINT)
        qcqp.suggest(RANDOM)    
        f_cd = qcqp.improve(ADMM)
        value = matrix(x.value.reshape(1,n))
        s = 0
        compteur = 0
        while s < q: 
            if np.sum((value-cU[s])**2, axis=1) >= RU[s]**2:
                compteur = compteur+1
            s = s +1
        if compteur == q:
            response == False
        
    return response

<img src="http://media.giphy.com/media/3o6Mb3Feec33LawNdm/giphy.gif"  width="300">

<a id="3"></a> 
 
# 3. QP vs. QCQP

## Exactness

In [325]:
#n = nb of variables
n = 2
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 10
diff = 1
epsilon = 50
(cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)

In [ ]:
plot(cI,RI,cU,RU,p,q, 100)

In [ ]:
#n = nb of variables
n = 2
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 2
diff = 1
epsilon = 10
diff = 1
nbQCQP = 0
nbQP = 0
nbTrue = 0

for i in range(0,100):
    (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
    Response = IsIncluded_PLOT(cI,RI,cU,RU,p,q)
    nbTrue = nbTrue + int(Response)
    nbQCQP = nbQCQP + int(IsIncluded_QCQP(cI,RI,cU,RU,n,p,q) == Response)   
    nbQP = nbQP + int(IsIncluded_QP(cI,RI,cU,RU,n,p,q) == Response)          
    print(i+1,nbTrue,nbQCQP,nbQP)

#### TABLE 1 OF THE PAPER : dimension 2

In [ ]:
#n = nb of variables
n = 2
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 3
diff = 1
epsilon = 10
nbQCQP = 0
nbQP = 0
nbTrue = 0

for i in range(0,100):
    (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
    j = 1
    while IsIncluded_PLOT(cI,RI,cU,RU,p,q) == True:
        (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
        j = j+1
        
    Response = IsIncluded_PLOT(cI,RI,cU,RU,p,q)
    nbTrue = nbTrue + int(Response)
    nbQCQP = nbQCQP + int(IsIncluded_QCQP(cI,RI,cU,RU,n,p,q) == Response)   
    nbQP = nbQP + int(IsIncluded_QP(cI,RI,cU,RU,n,p,q) == Response)          
    print(i+1,j,nbQCQP,nbQP)

In [ ]:
plot(cI,RI,cU,RU,p,q)

#### TABLE 1 OF THE PAPER : dimension > 2

In [ ]:
#n = nb of variables
n = 5
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 3
diff = 1
epsilon = 10
nbQCQP = 0
nbQP = 0
nbTrue = 0

for i in range(0,100):
    (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
    j = 1
    Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)
    while Response == True:
        (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
        Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)
        j = j+1
    
    nbQCQP = nbQCQP + int(IsIncluded_QCQP(cI,RI,cU,RU,n,p,q))   
    nbQP = nbQP + int(IsIncluded_QP(cI,RI,cU,RU,n,p,q))          
    print(i+1,j,nbQCQP,nbQP)

## computationnal time

#### TABLE 2 OF THE PAPER

In [365]:
import time
n = 5
#p = nb of elements in the intersection
p = 3
#p = nb of elements in the union
q = 3
diff = 1
epsilon = 10

In [ ]:
time_QP =0
time_QCQ = 0

for i in range(0,10):
    print(i)
    (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
    Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)    
    while Response == True:
        (cI,RI,cU,RU) = dataGenerator(epsilon,n,p,q,diff)
        Response = IsIncluded_QP(cI,RI,cU,RU,n,p,q)
        j = j+1
    
    time_startQP = time.clock()
    IsIncluded_QP(cI,RI,cU,RU,n,p,q)      
    time_QP = time_QP + (time.clock() - time_startQP)

    time_startQCQP = time.clock()
    IsIncluded_QCQP(cI,RI,cU,RU,n,p,q)   
    time_QCQP = time_QCQP + (time.clock() - time_startQCQP)   

print(time_QP,time_QCQP)
print(time_QCQP/time_QP)